Question 1 : Calculating Circular Orbits

In [1]:
# question 1.1 (conversion factor)

# constant in the CGS system according to the document
PI = 3.14159
G_CGS = 7.77430e-8 # the gravitational constant in CGS system
c_CGS = 2.99792e10 # the speed of light in CGS system
AU_CGS = 1.49598e13 # 1 AU in CGS system
M_CGS = 1.98854e33 # the mass of the sun in CGS system

# ratio between the escape velocities of an object escaping a planet with the mass of the sun and the radius of 1 AU

# CGS velocity to NEW velocity conversion factor, unit of centimeters per second
velocity_conversion_factor_0 = (G_CGS*M_CGS/AU_CGS)**(1/2)/(1*1/1)**(1/2) 

# ratio between the periods of an object orbiting a planet with the mass of the sun and the radius of 1 AU

# CGS time to NEW time conversion factor, unit of seconds
time_conversion_factor_0 = (1/velocity_conversion_factor_0)*(AU_CGS/1)

# rescale to units required for the question
velocity_conversion_factor_1 = velocity_conversion_factor_0*(1000/(1/100)) # units to kilometers per second
time_conversion_factor_1 = time_conversion_factor_0 # units not changed (seconds)

print("velocity conversion factor in kilometers per second = ",f'{velocity_conversion_factor_1:.12}')
print("time conversion factor in seconds = ",f'{time_conversion_factor_1:.12}')

velocity conversion factor in kilometers per second =  3.21465906271e+11
time conversion factor in seconds =  4653619.46886


In [ ]:
# question 1.2 (orbit integration)

In [ ]:
# question 1.3 (orbit integration)

In [ ]:
# question 1.4 ()